In [0]:
from sedona.spark import *

config = SedonaContext.builder() .\
    config('spark.jars.packages',
           'org.apache.sedona:sedona-spark-shaded-3.3_2.12:1.7.1,'
           'org.datasyslab:geotools-wrapper:1.7.1-28.5'). \
    getOrCreate()

In [0]:
from sedona.spark import *

sedona = SedonaContext.create(config)

In [0]:
df_greenspaces_bronze = spark.table("geospatial.greenspaces.greenspace_site") \
    .filter("function IN ('Play Space', 'Playing Field', 'Public Park Or Garden')")
df_greenspaces_bronze.createOrReplaceTempView("greenspace_site_bronze_vw")
df_greenspaces_bronze.count()


In [0]:
greenspace_site_covered = spark.sql("""
  SELECT 
    g1.id AS g1_id,
    g2.id AS g2_id,
    g1.function AS g1_function,
    g2.function AS g2_function,
    g2.distinctive_name_1 AS g2_name,
    g2.geometry AS geometry
  FROM greenspace_site_bronze_vw g1
  INNER JOIN greenspace_site_bronze_vw g2
    ON ST_CoveredBy(g1.geometry, g2.geometry)
   AND g1.id != g2.id
""")
greenspace_site_covered.cache()
greenspace_site_covered.createOrReplaceTempView("greenspace_site_covered_vw")
greenspace_site_covered.count()

In [0]:
greenspace_site_aggregated = spark.sql("""
  SELECT 
    g2_id AS id,
    concat_ws(', ', any_value(g2_function), collect_set(g1_function)) AS function,
    g2_name AS name,
    geometry
  FROM greenspace_site_covered_vw
  GROUP BY g2_id, g2_name, geometry
""")
greenspace_site_aggregated.cache()
greenspace_site_aggregated.createOrReplaceTempView("greenspace_site_aggregated_vw")
greenspace_site_aggregated.count()

In [0]:
greenspace_site_non_covered = spark.sql("""
  SELECT id, function, distinctive_name_1 AS name, geometry
  FROM greenspace_site_bronze_vw
  WHERE id NOT IN (SELECT g1_id FROM greenspace_site_covered_vw)
    AND id NOT IN (SELECT g2_id FROM greenspace_site_covered_vw)
""")
greenspace_site_non_covered.cache()
greenspace_site_non_covered.createOrReplaceTempView("greenspace_site_non_covered_vw")
greenspace_site_non_covered.count()


In [0]:
greenspace_site_merged = spark.sql("""
SELECT * FROM greenspace_site_aggregated_vw
UNION
SELECT * FROM greenspace_site_non_covered_vw""")

greenspace_site_merged.createOrReplaceTempView("greenspace_site_merged_vw")
greenspace_site_merged.count()

In [0]:
%sql
SELECT DISTINCT function
FROM greenspace_site_merged_vw


In [0]:
%sql
CREATE TABLE geospatial.greenspaces.parks_playgrounds (
    id STRING,
    function STRING,
    name STRING,
    geometry GEOMETRY
)
USING DELTA;

In [0]:
%sql
CREATE TABLE geospatial.greenspaces.parks_playgrounds
USING DELTA
AS
SELECT id, function, name, geometry
FROM 
WHERE function IN ('Play Space', 'Playing Field', 'Public Park Or Garden')